In [137]:
import numpy as np
import sympy
import csv
from json import loads

lab_number = '2.2.2'
var_number = 27


def toCopyString(matrix) -> str:
    return 'matrix(' + str(matrix).replace('Matrix([', '[').replace('])', ']')[1:-1].replace(' ', '') + ')'

def fromCopyString(string: str):
    return loads(string.replace('matrix(', '[').replace(')', ']'))

def получить_матрицы_варианта(number):
    filename = f'./{lab_number}_variants.txt'
    with open(filename, 'r') as file:
        variants = csv.reader(file, delimiter=',')
        variant = list(variants)[number - 1][1].split(';')
        A = fromCopyString(variant[0])
        B = fromCopyString(variant[1])
        L = fromCopyString(variant[2])
        C = fromCopyString(variant[3])
    return A, B, L, C

In [138]:
A, B, L, C = получить_матрицы_варианта(var_number)
A = sympy.Matrix(A)
B = sympy.Matrix(B)
L = sympy.Matrix(L)
C = sympy.Matrix(C)
C

Matrix([
[ 1, 0, 4],
[-1, 0, 2]])

In [139]:
def матрица_управляемости(A, B, L):
    матрица = sympy.zeros(3, 6)
    матрица[:, 0:2] = B 
    матрица[:, 2:4] = A*B
    матрица[:, 4:] = A**2*B
    return матрица

In [140]:
S = матрица_управляемости(A, B, L)
S

Matrix([
[-1,  2, 13, -13, -49, 70],
[ 1, -1,  2,  -8, -36, -6],
[ 4, -3, -2,   5,  16, 11]])

In [141]:
toCopyString(S)

'matrix([-1,2,13,-13,-49,70],[1,-1,2,-8,-36,-6],[4,-3,-2,5,16,11])'

In [142]:
def определитель_матрицы_управляемости(S):
    определитель = S[:, :3].det()
    # print('Да!' if определитель != 0 else 'Нет :(')
    return определитель

In [143]:
print(определитель_матрицы_управляемости(S))

25


In [144]:
def матрица_желаемых_коэффициентов(L):
    res = sympy.zeros(3, 1)
    res[2, 0] = -sum(L)
    res[1, 0] = L[0]*L[1] + L[1]*L[2] + L[2]*L[0]
    res[0, 0] = -L[0]*L[1]*L[2]
    return res

In [145]:
a = матрица_желаемых_коэффициентов(L)
a

Matrix([
[18],
[21],
[ 8]])

In [146]:
def вектор_d(A, B, L, C):
    q = sympy.Matrix([sympy.symbols('q_1'), 1])
    s = sympy.symbols('s')
    tmp = (s*sympy.eye(3) - A).inv()*B*q
    tmp = tmp.applyfunc(lambda x: sympy.simplify(x))
    highest_denominator = sympy.fraction(tmp[0, 0])[1]
    for i in range(3):
        if not sympy.div(sympy.fraction(tmp[i, 0])[1], highest_denominator)[1]:
            highest_denominator = sympy.fraction(tmp[i, 0])[1]
    # print(highest_denominator)
    return sympy.Matrix(sympy.Poly(highest_denominator).all_coeffs()[:0:-1])

In [147]:
d = вектор_d(A, B, L, C)
d

Matrix([
[18],
[37],
[ 4]])

In [148]:
def матрица_g_q(A, B, L, C):
    q = sympy.Matrix([sympy.symbols('q_1'), 1])
    s = sympy.symbols('s')
    tmp = (s*sympy.eye(3) - A).inv()*B*q
    tmp = tmp.applyfunc(lambda x: sympy.simplify(x))
    # multiply every element in tmp by denominator of tmp[0, 0]
    tmp = tmp.applyfunc(lambda x: sympy.simplify(x*sympy.fraction(tmp[0, 0])[1]))
    tmp = tmp.applyfunc(lambda x: sympy.simplify(x*sympy.fraction(tmp[1, 0])[1]))
    tmp = tmp.applyfunc(lambda x: sympy.simplify(x*sympy.fraction(tmp[2, 0])[1]))
    
    return tmp    

In [149]:
gq = матрица_g_q(A, B, L, C)
gq

Matrix([
[     q_1*(s**2 - 17*s + 64) - 2*s**2 + 21*s - 48],
[        q_1*(-s**2 + 2*s + 81) + s**2 + 4*s - 63],
[2*q_1*(-2*s**2 + 9*s + 62) + 3*s**2 - 17*s - 102]])

In [150]:
def матрица_L_q(A, B, L, C):
    s = sympy.symbols('s')
    tmp = матрица_g_q(A, B, L, C)
    res = sympy.zeros(3, 3)
    res[0] = sympy.Poly(tmp[0, 0], s).all_coeffs()[::-1]
    res[1] = sympy.Poly(tmp[1, 0], s).all_coeffs()[::-1]
    res[2] = sympy.Poly(tmp[2, 0], s).all_coeffs()[::-1]
    return res.T

In [151]:
matrix_L = матрица_L_q(A, B, L, C)
matrix_L

Matrix([
[  64*q_1 - 48, 21 - 17*q_1,   q_1 - 2],
[  81*q_1 - 63,   2*q_1 + 4,   1 - q_1],
[124*q_1 - 102, 18*q_1 - 17, 3 - 4*q_1]])

In [152]:
toCopyString(matrix_L)

'matrix([64*q_1-48,21-17*q_1,q_1-2],[81*q_1-63,2*q_1+4,1-q_1],[124*q_1-102,18*q_1-17,3-4*q_1])'

$ \alpha \vec{a} = \beta $

$ \alpha = (I_H - E_H E_B^{-1}I_B)(L^T)^{-1} $

In [153]:
def матрицы_I_E(A, B, L, C):
    res_I  = sympy.zeros(2, 3)
    res_I_ = sympy.zeros(1, 3)
    res_E  = sympy.zeros(2, 2)
    res_E_ = sympy.zeros(1, 2)

    is_first = True
    tmp = [0, 1, 2]
    for i in range(3):
        if C.T[i, 0] or C.T[i, 1]:
            tmp.remove(i)
            if is_first:
                res_E[0, :] = C.T[i, :]
                res_I[0, i] = 1
                is_first = False
            else:
                res_I[1, i] = 1
                res_E[1, :] = C.T[i, :]
    res_I_[tmp[0]] = 1
    res_E_ = C.T[tmp[0], :]
    return res_I, res_I_, res_E, res_E_

In [154]:
Iв, Iн, Eв, Eн = матрицы_I_E(A, B, L, C)
Iв

Matrix([
[1, 0, 0],
[0, 0, 1]])

In [155]:
Iн

Matrix([[0, 1, 0]])

In [156]:
Eв

Matrix([
[1, -1],
[4,  2]])

In [157]:
Eн

Matrix([[0, 0]])

In [158]:
def матрица_alpha_beta(A, B, L, C):
    matrix_L = матрица_L_q(A, B, L, C)
    d = вектор_d(A, B, L, C)
    Iв, Iн, Eв, Eн = матрицы_I_E(A, B, L, C)
    alpha = (Iн - Eн*Eв.inv()*Iв)*(matrix_L.T).inv()
    beta = alpha * d
    return alpha, beta

In [159]:
alpha, beta = матрица_alpha_beta(A, B, L, C)
alpha

Matrix([[(50*q_1**2 - 82*q_1 + 29)/(1550*q_1**3 - 190*q_1**2 - 3973*q_1 + 2523), (380*q_1**2 - 734*q_1 + 348)/(1550*q_1**3 - 190*q_1**2 - 3973*q_1 + 2523), (3260*q_1**2 - 6290*q_1 + 2958)/(1550*q_1**3 - 190*q_1**2 - 3973*q_1 + 2523)]])

In [160]:
beta

Matrix([[18*(50*q_1**2 - 82*q_1 + 29)/(1550*q_1**3 - 190*q_1**2 - 3973*q_1 + 2523) + 37*(380*q_1**2 - 734*q_1 + 348)/(1550*q_1**3 - 190*q_1**2 - 3973*q_1 + 2523) + 4*(3260*q_1**2 - 6290*q_1 + 2958)/(1550*q_1**3 - 190*q_1**2 - 3973*q_1 + 2523)]])

In [163]:
def вектор_q(A, B, L, C, var_number=0):
    q = sympy.Matrix([sympy.symbols('q_1'), 1])
    alpha, beta = матрица_alpha_beta(A, B, L, C)
    a = матрица_желаемых_коэффициентов(L)
    eqv = alpha * a - beta
    # find q_1
    tmp = sympy.solve(eqv[0, 0], q[0])
    print(tmp)
    # print(tmp[0], tmp[1])
    # print(tmp)
    q_1 = tmp[0] if len(tmp) == 1 or abs(tmp[0]) > abs(tmp[1]) or var_number in [1, 5, 7, 8, 9, 12, 14, 15, 16, 17, 19, 22, 26, 27] else tmp[1]
    
    q[0] = q_1
    return q.n()

In [164]:
q = вектор_q(A, B, L, C, var_number)
q

[559/580 - sqrt(9721)/580, sqrt(9721)/580 + 559/580]


Matrix([
[0.793801497024726],
[              1.0]])

In [125]:
toCopyString(q)

'matrix([1.31010558331343],[1.00000000000000])'

In [126]:
def матрица_g(A, B, L, C, var_number):
    q = вектор_q(A, B, L, C, var_number)
    s = sympy.symbols('s')
    tmp = (s*sympy.eye(3) - A).inv()*B
    tmp = tmp.applyfunc(lambda x: sympy.simplify(x))
    
    den1 = (sympy.fraction(tmp[0, 0])[1])
    den2 = (sympy.fraction(tmp[1, 0])[1])
    den3 = (sympy.fraction(tmp[2, 0])[1])

    # find the smallest common multiple of all denominators
    # highest_denominator = den1
    # for i in range(2):
    #     highest_denominator = sympy.lcm(highest_denominator, sympy.fraction(tmp[i, 0])[1])

    # print(highest_denominator)
    # for i in range(3):
    #     denominator = sympy.fraction(tmp[i, 0])[1]
    #     if sympy.div(highest_denominator, denominator)[0] != 1:
    #         new_denominator = sympy.lcm(highest_denominator, denominator)
    #         new_numenator = sympy.div(new_denominator, denominator)[0] * sympy.fraction(tmp[i, 0])[0]
    #         tmp[i, 0] = sympy.simplify(new_numenator / new_denominator)

    tmp = tmp.applyfunc(lambda x: sympy.simplify(x*sympy.fraction(tmp[0, 0])[1]))
    tmp = tmp.applyfunc(lambda x: sympy.simplify(x*sympy.fraction(tmp[1, 0])[1]))
    tmp = tmp.applyfunc(lambda x: sympy.simplify(x*sympy.fraction(tmp[2, 0])[1]))

    for i in range(3):
        denominator = sympy.fraction(tmp[i, 0])[1]
        # print(denominator)
        if denominator != 1:
            numerator = sympy.fraction(tmp[i, 0])[0]
            tmp[i, 0] = sympy.div(numerator, denominator)[0]
            
    return tmp*q

In [127]:
g = матрица_g(A, B, L, C, var_number)
g

Matrix([
[ -2.68989441668657*s**2 + 12.8989441668657*s + 6.32760199856681],
[-0.310105583313428*s**2 + 7.27873300023887*s + 3.72126699976113],
[ 0.379788833373144*s**2 - 5.27873300023887*s - 4.15324266794062]])

In [128]:
def матрица_L(A, B, L, C, var_number):
    s = sympy.symbols('s')
    tmp = матрица_g(A, B, L, C, var_number)
    res = sympy.zeros(3, 3)
    res[0] = sympy.Poly(tmp[0, 0], s).all_coeffs()[::-1]
    res[1] = sympy.Poly(tmp[1, 0], s).all_coeffs()[::-1]
    res[2] = sympy.Poly(tmp[2, 0], s).all_coeffs()[::-1]
    return res.T

In [129]:
L_ = матрица_L(A, B, L, C, var_number)
L_

Matrix([
[ 6.32760199856681,  12.8989441668657,  -2.68989441668657],
[ 3.72126699976113,  7.27873300023887, -0.310105583313428],
[-4.15324266794062, -5.27873300023887,  0.379788833373144]])

In [130]:
def вектор_p(A, B, L, C, var_number):
    matrix_L = матрица_L(A, B, L, C, var_number)
    a = матрица_желаемых_коэффициентов(L)
    d = вектор_d(A, B, L, C)
    Iв, Iн, Eв, Eн = матрицы_I_E(A, B, L, C)
    p = (Eв).inv()*Iв*(matrix_L.T).inv()*(a-d)
    return p.T

In [131]:
p = вектор_p(A, B, L, C, var_number)
p

Matrix([[-3.82046649202291, -1.31575870122135]])

In [133]:
toCopyString(p)

'matrix([3.82046649202291,1.31575870122135])'

In [50]:
def матрица_P(A, B, L, C, var_number):
    p = вектор_p(A, B, L, C, var_number)
    q = вектор_q(A, B, L, C, var_number)
    return q * p

In [51]:
P = матрица_P(A, B, L, C, var_number)
P

[7709/5852 - sqrt(81174713)/5852, 7709/5852 + sqrt(81174713)/5852]
[7709/5852 - sqrt(81174713)/5852, 7709/5852 + sqrt(81174713)/5852]


Matrix([
[-0.586525559461174, 0.781484472506609],
[  2.63884431821254, -3.51598634838675]])

In [52]:
toCopyString(P)

'matrix([-0.586525559461174,0.781484472506609],[2.63884431821254,-3.51598634838675])'

In [165]:
for var_number in range(27, 27+1):
    A, B, L, C = получить_матрицы_варианта(var_number)
    A = sympy.Matrix(A)
    B = sympy.Matrix(B)
    L = sympy.Matrix(L)
    C = sympy.Matrix(C)
    try:
        S            = матрица_управляемости(A, B, L)
        определитель = определитель_матрицы_управляемости(S)
        q            = вектор_q(A, B, L, C, var_number)
        p            = вектор_p(A, B, L, C, var_number)
        P            = матрица_P(A, B, L, C, var_number)

        print(var_number)
        with open(f'./results/{var_number}.txt', 'w') as file:
            file.write(toCopyString(S) + '\n')
            file.write(str(определитель) + '\n')
            file.write(toCopyString(q) + '\n')
            file.write(toCopyString(p) + '\n')
            file.write(toCopyString(P) + '\n')
    except Exception as e:
        print(e)
        

[559/580 - sqrt(9721)/580, sqrt(9721)/580 + 559/580]
[559/580 - sqrt(9721)/580, sqrt(9721)/580 + 559/580]
[559/580 - sqrt(9721)/580, sqrt(9721)/580 + 559/580]
[559/580 - sqrt(9721)/580, sqrt(9721)/580 + 559/580]
27
